Download the FER-2013 dataset: The FER-2013 dataset is available for download from Kaggle and other sources. It contains 35,887 grayscale images of faces labeled with one of seven emotions (angry, disgust, fear, happy, sad, surprise, neutral).  

Prepare the dataset: You'll need to preprocess the images to prepare them for training. This may involve resizing, cropping, and normalization.  

Build the model: There are several deep learning frameworks that you can use to build an emotion classification model, such as TensorFlow, PyTorch, or Keras. You can choose to build your own custom model architecture or use a pre-trained model like VGG, ResNet, or Inception.
  
  Train the model: Split the dataset into training and validation sets and use the training data to train the model. Monitor the model's performance on the validation set to prevent overfitting.
  
  Evaluate the model: Use the test set to evaluate the model's performance on unseen data. Calculate metrics such as accuracy, precision, recall, and F1 score to measure the model's performance.
  
  Fine-tune the model: If the model is not performing well enough, you can try fine-tuning the model by adjusting the model architecture, hyperparameters, or training data.
  
  Deploy the model: Once you're satisfied with the model's performance, you can deploy it in a production environment to classify emotions in real-time.

> resizing, cropping, and normalization.

In [32]:
import numpy as np
import cv2

def preprocess_image(image_path, target_size):
    """
    Preprocess a JPEG image for training an emotion classification model.

    Parameters:
        image_path (str): Path to the JPEG image file.
        target_size (tuple): Desired size of the preprocessed image.

    Returns:
        np.ndarray: Preprocessed image as a NumPy array.
    """
    # Load the image using OpenCV.
    image = cv2.imread(image_path)

    # Convert the image to grayscale.
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Resize the image to a larger size while maintaining aspect ratio.
    height, width = image.shape
    if width > height:
        scale = target_size[0] / width
    else:
        scale = target_size[1] / height
    image = cv2.resize(image, None, fx=scale, fy=scale)

    # Crop the center of the resized image to the target size.
    height, width = image.shape
    y1 = (height - target_size[1]) // 2
    y2 = y1 + target_size[1]
    x1 = (width - target_size[0]) // 2
    x2 = x1 + target_size[0]
    image = image[y1:y2, x1:x2]

    # Normalize the pixel values to the range [-1, 1].
    image = (image / 255.0) * 2.0 - 1.0

    # Convert the image to a NumPy array and add a batch dimension.
    image = np.expand_dims(image, axis=0)

    return image


having error importing cv2 :sudo apt-get update /
sudo apt-get install libgl1-mesa-glx


In [35]:
# test preprocess_image on training image
image = preprocess_image('archive/train/angry/Training_3908.jpg', target_size=(224, 224))

In [40]:
image.shape

(1, 224, 224)

> train our own model using the VGG architecture,This approach allows you to benefit from the already learned features by the VGG model and adapt it to specific problem.

In [45]:
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Load the VGG16 model without the top layers
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the base model layers (optional)
for layer in base_model.layers:
    layer.trainable = False

In [46]:
# Add custom layers on top of the base model
x = base_model.output
x = Flatten()(x)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(512, activation='relu')(x)
output_layer = Dense(7, activation='softmax')(x)  # Change to 7 classes for emotions


In [49]:
# Create the final model
model = Model(inputs=base_model.input, outputs=output_layer)

# Compile the model
model.compile(optimizer=Adam('learning_rate'==1e-4), loss='categorical_crossentropy', metrics=['accuracy'])

# Create data generators for training and validation data
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

valid_datagen = ImageDataGenerator(rescale=1./255)

In [50]:
# Create data generators for training and validation data
train_data_dir = 'archive/train/'
valid_data_dir = 'archive/test/'

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(224, 224),
    batch_size=32,  # Change to your desired batch size
    class_mode='categorical',
    classes=['fear', 'sad', 'surprise', 'happy', 'angry', 'neutral', 'disgust'])

valid_generator = valid_datagen.flow_from_directory(
    valid_data_dir,
    target_size=(224, 224),
    batch_size=32,  # Change to your desired batch size
    class_mode='categorical',
    classes=['fear', 'sad', 'surprise', 'happy', 'angry', 'neutral', 'disgust'])

# Train the model
model.fit(
    train_generator,
    epochs=50,  # Change to your desired number of epochs
    validation_data=valid_generator,
    steps_per_epoch=len(train_generator),
    validation_steps=len(valid_generator))

Found 28709 images belonging to 7 classes.
Found 7178 images belonging to 7 classes.
Epoch 1/50


2023-04-01 17:52:54.397630: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


898/898 [==============================] - ETA: 0s - loss: 2.1975 - accuracy: 0.1693

2023-04-01 18:35:49.311196: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


898/898 [==============================] - 3178s 4s/step - loss: 2.1975 - accuracy: 0.1693 - val_loss: 2.0477 - val_accuracy: 0.1719
Epoch 2/50
 61/898 [=>............................] - ETA: 39:46 - loss: 2.2230 - accuracy: 0.1603

KeyboardInterrupt: 

In [43]:
# this. is for the GPU 
lspci | grep -i vga

SyntaxError: invalid syntax (2569466510.py, line 1)

In [51]:
!pip install transformers
!pip install -U tensorflow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 63.7 MB/s eta 0:00:00:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.1/77.1 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 769.6/769.6 kB 23.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 60.2 MB/s eta 0:00:00:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 kB 8.6 MB/s eta 0:00:00


In [52]:
import numpy as np
import tensorflow as tf
from transformers import ViTFeatureExtractor, TFAutoModelForImageClassification

# Load the ViT model and feature extractor
feature_extractor = ViTFeatureExtractor.from_pretrained("google/vit-base-patch16-224-in21k")
model = TFAutoModelForImageClassification.from_pretrained("google/vit-base-patch16-224-in21k")


/home/codespace/.python/current/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/codespace/.python/current/lib/python3.10/site-packages/transformers/models/vit/feature_extraction_vit.py:28: FutureWarning: The class ViTFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ViTImageProcessor instead.
  warnings.warn(


RuntimeError: Failed to import transformers.models.vit.modeling_tf_vit because of the following error (look up to see its traceback):
Failed to import transformers.data.data_collator because of the following error (look up to see its traceback):
No module named 'numpy.testing'